In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [80]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle['daily_outage_dfs'][6]

(datetime.datetime(2022, 9, 13, 14, 9, 10, 133302, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                     timeStamp  value  fuelType
 0   2022-09-13 00:00:00-06:00    712  Intertie
 1   2022-09-14 00:00:00-06:00      0  Intertie
 2   2022-09-15 00:00:00-06:00      0  Intertie
 3   2022-09-16 00:00:00-06:00      0  Intertie
 4   2022-09-17 00:00:00-06:00      0  Intertie
 ..                        ...    ...       ...
 104 2022-12-26 00:00:00-07:00      0      Wind
 105 2022-12-27 00:00:00-07:00      0      Wind
 106 2022-12-28 00:00:00-07:00      0      Wind
 107 2022-12-29 00:00:00-07:00      0      Wind
 108 2022-12-30 00:00:00-07:00      0      Wind
 
 [1224 rows x 3 columns])

In [45]:
default_pickle

(datetime.datetime(2022, 9, 13, 14, 5, 41, 529377, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                     timeStamp  value  fuelType
 0   2022-09-13 00:00:00-06:00    712  Intertie
 1   2022-09-14 00:00:00-06:00      0  Intertie
 2   2022-09-15 00:00:00-06:00      0  Intertie
 3   2022-09-16 00:00:00-06:00      0  Intertie
 4   2022-09-17 00:00:00-06:00      0  Intertie
 ..                        ...    ...       ...
 104 2022-12-26 00:00:00-07:00      0      Wind
 105 2022-12-27 00:00:00-07:00      0      Wind
 106 2022-12-28 00:00:00-07:00      0      Wind
 107 2022-12-29 00:00:00-07:00      0      Wind
 108 2022-12-30 00:00:00-07:00      0      Wind
 
 [1224 rows x 3 columns])

In [39]:
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
daily

,timeStamp,value,fuelType
0,2022-09-12 00:00:00-06:00,750,Intertie
1,2022-09-13 00:00:00-06:00,0,Intertie
2,2022-09-14 00:00:00-06:00,0,Intertie
3,2022-09-15 00:00:00-06:00,0,Intertie
4,2022-09-16 00:00:00-06:00,0,Intertie
...,...,...,...
86,2022-12-07 00:00:00-07:00,0,Wind
87,2022-12-08 00:00:00-07:00,0,Wind
88,2022-12-09 00:00:00-07:00,0,Wind
89,2022-12-10 00:00:00-07:00,0,Wind


In [154]:
df1 = pd.DataFrame({'fuel':['oil','gas','solar','wind'],'val':[10,20,30,40],'time':[1,1,1,1]})
df2 = pd.DataFrame({'fuel':['oil','solar'],'val':[15,34],'time':[2,2]})
df3 = pd.merge(df1,df2, on='fuel', how='outer')
df3['merge']=np.where(df3['time_y']>df3['time_x'],df3['val_y'],df3['val_x'])
df3

,fuel,val_x,time_x,val_y,time_y,merge
0,oil,10,1,15.0,2.0,15.0
1,gas,20,1,NaN,NaN,20.0
2,solar,30,1,34.0,2.0,34.0
3,wind,40,1,NaN,NaN,40.0


In [151]:
np.where(df1>df2,df2)

ValueError: Can only compare identically-labeled DataFrame objects